**Axial Force- Bending Moment Interaction Curve**

Inputs for the code

In [ ]:
import pandas as pd
from math import pi
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from matplotlib.patches import Rectangle
import seaborn as sns

class BeamSection:
    def __init__(self):

        print("\n--- MATERIAL PROPERTIES ---")
        self.fck = float(input("Enter fck (MPa): "))
        self.fyk = float(input("Enter fyk (MPa): "))
        self.Es = float(input("Enter Es (MPa): "))
        self.Ec = float(input("Enter Ec (MPa): "))
        self.SFc = float(input("Enter SFc (Safety factor for concrete): "))
        self.SFs = float(input("Enter SFs (Safety factor for steel): "))
        self.eta = 1 - (self.fck - 50)/200
        print(f"Computed η (eta) = {self.eta:.4f}")
        self.lamda = 0.80 - (self.fck - 50)/400
        print(f"Computed λ (lamda) = {self.lamda:.4f}")
        self.epsilon_co = 0.002
        print(f"Minimum allowable strain in concrete, ε_co  = {self.epsilon_co:4f}")
        self.epsilon_cu = 0.0035
        print(f"Maximum allowable strain in concrete, ε_cu = {self.epsilon_cu:4f}")
        self.epsilon_su = 0.01
        print(f"Maximum allowable strain in steel, ε_su = {self.epsilon_cu:4f}")
        self.epsilon_yd = (self.fyk/self.SFs)/self.Es
        print(f"Yield strain in concrete, ε_yd = {self.epsilon_yd:4f}")

        print("\n--- SECTION PROPERTIES ---")
        self.D = float(input("Enter total depth D (mm): "))
        self.b = float(input("Enter width b (mm): "))
        self.d = float(input("Enter tension cover d (mm): "))
        self.d_prime = float(input("Enter compression cover d' (mm): "))


        print("\n --- REINFORCEMENT DETAILS ---")
        self.dst = float(input("Enter diameter of bars in tension zone: "))
        self.nst = float(input("Enter number of bars in tension zone: "))
        self.dsc = float(input("Enter diameter of bars in compression zone: "))
        self.nsc = float(input("Enter number of bars in compression zone: "))
        self.Ast= self.nst * pi * 0.25 * self.dst**2
        print(f"Area of steel in tension zone = {self.Ast:.4f} mm^2")
        self.Asc = self.nsc * pi * 0.25 * self.dsc**2
        print(f"Area of steel in compression zone = {self.Asc:.4f} mm^2")

        print("\nAll inputs recorded successfully.\n")

beam = BeamSection()

Domain Categorization and Domain wise formulation

In [ ]:
import pandas as pd
from math import pi
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from matplotlib.patches import Rectangle
import seaborn as sns


class NeutralAxisDomain:

    def __init__(self, beam_section_obj):
        # Store necessary properties from beam_section_obj
        self.epsilon_cu = beam_section_obj.epsilon_cu
        self.epsilon_co = beam_section_obj.epsilon_co
        self.epsilon_su = beam_section_obj.epsilon_su
        self.epsilon_yd = beam_section_obj.epsilon_yd
        self.D = beam_section_obj.D
        self.b = beam_section_obj.b
        self.d = beam_section_obj.d
        self.d_prime = beam_section_obj.d_prime
        self.Es = beam_section_obj.Es
        self.Ast = beam_section_obj.Ast
        self.Asc = beam_section_obj.Asc
        # Added missing material properties
        self.lamda = beam_section_obj.lamda
        self.eta = beam_section_obj.eta
        self.fck = beam_section_obj.fck
        self.SFc = beam_section_obj.SFc

        self.x1 = self.compute_x1()
        self.x2 = self.compute_x2()

    def compute_x1(self):
        x1 = (self.epsilon_cu * (self.D - self.d))/(self.epsilon_cu + self.epsilon_su)
        return x1


    def compute_x2(self):
        x2 = (self.epsilon_cu * (self.D - self.d))/(self.epsilon_cu + self.epsilon_yd)
        return x2

    def get_domain(self, x):
        if x <= 0:
            return "Domain 1"                     # Domain 1: (-∞ , 0], includes x=0 where NA is at top

        elif 0 < x < self.x1:
            return "Domain 2"                     # Domain 2: (0 , x'], x > 0 to avoid division by zero

        elif self.x1 <= x < self.x2:
            return "Domain 3"                     # Domain 3: [x' , x'']

        elif self.x2 <= x < (self.D - self.d):
            return "Domain 4"                     # Domain 4: [x'' , D-d]

        elif (self.D - self.d) <= x < self.D:
            return "Domain 4a"                  # Domain 4a: [D-d , D]

        else:
            return "Domain 5"                     # Domain 5: [D , +∞]

    def compute_domain1_values(self, x):
        epsilon_c = 0 # Strain at extreme compression fiber (top) is 0 as concrete is not in compression or NA is at top.
        epsilon_st = - self.epsilon_su # Strain in tension steel is at ultimate tensile strain (fixed as per original code's intent)

        # Calculate epsilon_sc based on linear strain distribution and epsilon_st
        # If x is NA, strain at depth y is proportional to (y - x)
        # epsilon_sc / (self.d_prime - x) = epsilon_st / (self.d - x)
        # Ensure (self.d - x) is not zero. Since self.d > 0 and x <= 0, (self.d - x) will always be positive.
        epsilon_sc = epsilon_st * (self.d_prime - x) / (self.d - x)

        # Calculate Nst (Axial force in tension steel)
        # Stress in tension steel
        if abs(epsilon_st) > self.epsilon_yd:
          if epsilon_st < 0:
            Nst = -(self.epsilon_yd * self.Es * self.Ast) / 1000
          else:
            Nst =  (self.epsilon_yd * self.Es * self.Ast) / 1000
        else:
          Nst = (epsilon_st * self.Es * self.Ast) / 1000

        # Calculate Nsc (Axial force in compression steel)
        # Stress in compression steel
        if abs(epsilon_sc) > self.epsilon_yd:
          if epsilon_sc < 0:
            Nsc = -(self.epsilon_yd * self.Es * self.Asc) / 1000
          else:
            Nsc =  (self.epsilon_yd * self.Es * self.Asc) / 1000
        else:
          Nsc = (epsilon_st * self.Es * self.Ast) / 1000

        # Calculate Nc (Axial force in concrete)
        if x < self.D:
          lamda = (x * self.lamda)/ self.D
        else:
          lamda = 1 - (1-self.lamda)*(self.d/x)
        Nc = 0 # No concrete compression for Domain 1 (x <= 0)

        # Calculate total Axial force Nd and Moment Md
        Nd = Nst + Nsc + Nc
        # Md formula is retained as per original structure, but its correctness needs further review.
        Md = ((Nsc*(self.D - self.d- self.d_prime)+ Nc*(self.D-self.d-(lamda*self.D*0.5))-Nd*(self.D*0.5 - self.d))*1000)/1000000

        return {
            "ε_c": epsilon_c,
            "ε_sc": epsilon_sc,
            "ε_st": epsilon_st,
            "Nst": Nst,
            "Nsc": Nsc,
            "Nc": Nc,
            "Nd": Nd,
            "lamda": lamda,
            "Md": Md
        }

    def compute_domain2_values(self, x):
        # Domain 2: 0 < x < self.x1
        # Strain at extreme compression fiber (top) is epsilon_cu
        epsilon_c = self.epsilon_su*(x/(self.D-self.d-x))

        # Strain in compression steel: epsilon_sc = epsilon_cu * (x - d_prime) / x
        epsilon_sc = -self.epsilon_su *((self.d_prime -x)/(self.D - self.d-x))

        # Strain in tension steel: epsilon_st = epsilon_cu * (d - x) / x
        epsilon_st = - self.epsilon_su

        # Calculate Nst (Axial force in tension steel)
        # Stress in tension steel
        if abs(epsilon_st) > self.epsilon_yd:
          if epsilon_st < 0:
            Nst = -(self.epsilon_yd * self.Es * self.Ast) / 1000
          else:
            Nst =  (self.epsilon_yd * self.Es * self.Ast) / 1000
        else:
          Nst = (epsilon_st * self.Es * self.Ast) / 1000

        # Calculate Nsc (Axial force in compression steel)
        # Stress in compression steel
        if abs(epsilon_sc) > self.epsilon_yd:
          if epsilon_sc < 0:
            Nsc = -(self.epsilon_yd * self.Es * self.Asc) / 1000
          else:
            Nsc =  (self.epsilon_yd * self.Es * self.Asc) / 1000
        else:
          Nsc = (epsilon_sc * self.Es * self.Asc) / 1000

        # Calculate Nc (Axial force in concrete)
        # Corrected Nc calculation to use x (neutral axis depth) for compression block depth (lambda * x)
        if x < self.D:
          lamda = (x * self.lamda)/ self.D
        else:
          lamda = 1 - (1-self.lamda)*(self.d/x)
        Nc = (self.eta * (self.fck / self.SFc) * self.b * (self.lamda * x)) / 1000

        # Calculate total Axial force Nd and Moment Md
        Nd = Nst + Nsc + Nc
        # Md formula is retained as per original structure, but its correctness needs further review.
        Md = ((Nsc*(self.D - self.d - self.d_prime))+ (Nc * (self.D - self.d - lamda*self.D*0.5))- (Nd*(self.D*0.5-self.d)))/1000

        return {
            "ε_c": epsilon_c,
            "ε_sc": epsilon_sc,
            "ε_st": epsilon_st,
            "Nst": Nst,
            "Nsc": Nsc,
            "Nc": Nc,
            "Nd": Nd,
            "lamda": lamda,
            "Md": Md
        }

    def compute_domain3_values(self, x):
        # Domain 3: self.x1 < x < self.x2
        # Strain at extreme compression fiber (top) is epsilon_cu
        epsilon_c = self.epsilon_cu

        # Strain in compression steel:
        epsilon_sc = self.epsilon_cu *((x - self.d_prime)/(x))

        # Strain in tension steel:
        epsilon_st = - self.epsilon_cu * ((self.D - self.d - x)/x)

        # Calculate Nst (Axial force in tension steel)
        # Stress in tension steel
        if abs(epsilon_st) > self.epsilon_yd:
          if epsilon_st < 0:
            Nst = -(self.epsilon_yd * self.Es * self.Ast) / 1000
          else:
            Nst =  (self.epsilon_yd * self.Es * self.Ast) / 1000
        else:
          Nst = (epsilon_st * self.Es * self.Ast) / 1000

        # Calculate Nsc (Axial force in compression steel)
        # Stress in compression steel
        if abs(epsilon_sc) > self.epsilon_yd:
          if epsilon_sc < 0:
            Nsc = -(self.epsilon_yd * self.Es * self.Asc) / 1000
          else:
            Nsc =  (self.epsilon_yd * self.Es * self.Asc) / 1000
        else:
          Nsc = (epsilon_sc * self.Es * self.Asc) / 1000

        # Calculate Nc (Axial force in concrete)
        # Corrected Nc calculation to use x (neutral axis depth) for compression block depth (lambda * x)
        if x < self.D:
          lamda = (x * self.lamda)/ self.D
        else:
          lamda = 1 - (1-self.lamda)*(self.d/x)
        Nc = (self.eta * (self.fck / self.SFc) * self.b * (self.lamda * x)) / 1000

        # Calculate total Axial force Nd and Moment Md
        Nd = Nst + Nsc + Nc
        # Md formula is retained as per original structure, but its correctness needs further review.

        Md = ((Nsc*(self.D - self.d - self.d_prime))+ (Nc * (self.D - self.d - lamda*self.D*0.5))- (Nd*(self.D*0.5-self.d)))/1000

        return {
            "ε_c": epsilon_c,
            "ε_sc": epsilon_sc,
            "ε_st": epsilon_st,
            "Nst": Nst,
            "Nsc": Nsc,
            "Nc": Nc,
            "Nd": Nd,
            "lamda": lamda,
            "Md": Md
        }
    def compute_domain4_values(self, x):
        # Domain 4: self.x2 < x < self.D - self.d
        # Strain at extreme compression fiber (top) is epsilon_cu
        epsilon_c = self.epsilon_cu

        # Strain in compression steel:
        epsilon_sc = self.epsilon_cu *((x - self.d_prime)/(x))

        # Strain in tension steel:
        epsilon_st = - self.epsilon_cu * ((self.D - self.d - x)/x)

        # Calculate Nst (Axial force in tension steel)
        # Stress in tension steel
        if abs(epsilon_st) > self.epsilon_yd:
          if epsilon_st < 0:
            Nst = -(self.epsilon_yd * self.Es * self.Ast) / 1000
          else:
            Nst =  (self.epsilon_yd * self.Es * self.Ast) / 1000
        else:
          Nst = (epsilon_st * self.Es * self.Ast) / 1000

        # Calculate Nsc (Axial force in compression steel)
        # Stress in compression steel
        if abs(epsilon_sc) > self.epsilon_yd:
          if epsilon_sc < 0:
            Nsc = -(self.epsilon_yd * self.Es * self.Asc) / 1000
          else:
            Nsc =  (self.epsilon_yd * self.Es * self.Asc) / 1000
        else:
          Nsc = (epsilon_sc * self.Es * self.Asc) / 1000

        # Calculate Nc (Axial force in concrete)
        # Corrected Nc calculation to use x (neutral axis depth) for compression block depth (lambda * x)
        if x < self.D:
          lamda = (x * self.lamda)/ self.D
        else:
          lamda = 1 - ((1-self.lamda)*(self.D/x))
        Nc = (self.eta * (self.fck / self.SFc) * self.b * (self.lamda * x)) / 1000

        # Calculate total Axial force Nd and Moment Md
        Nd = Nst + Nsc + Nc
        # Md formula is retained as per original structure, but its correctness needs further review.
        Md = ((Nsc*(self.D - self.d - self.d_prime))+ (Nc * (self.D - self.d - lamda*self.D*0.5))- (Nd*(self.D*0.5-self.d)))/1000

        return {
            "ε_c": epsilon_c,
            "ε_sc": epsilon_sc,
            "ε_st": epsilon_st,
            "Nst": Nst,
            "Nsc": Nsc,
            "Nc": Nc,
            "Nd": Nd,
            "lamda": lamda,
            "Md": Md
        }
    def compute_domain4a_values(self, x):
        # Domain 4: self.D - self.d < x < self.D
        # Strain at extreme compression fiber (top) is epsilon_cu
        epsilon_c = self.epsilon_cu

        # Strain in compression steel:
        epsilon_sc = self.epsilon_cu *((x - self.d_prime)/(x))

        # Strain in tension steel:
        epsilon_st = - self.epsilon_cu * ((self.D - self.d - x)/x)

        # Calculate Nst (Axial force in tension steel)
        # Stress in tension steel
        if abs(epsilon_st) > self.epsilon_yd:
          if epsilon_st < 0:
            Nst = -(self.epsilon_yd * self.Es * self.Ast) / 1000
          else:
            Nst =  (self.epsilon_yd * self.Es * self.Ast) / 1000
        else:
          Nst = (epsilon_st * self.Es * self.Ast) / 1000

        # Calculate Nsc (Axial force in compression steel)
        # Stress in compression steel
        if abs(epsilon_sc) > self.epsilon_yd:
          if epsilon_sc < 0:
            Nsc = -(self.epsilon_yd * self.Es * self.Asc) / 1000
          else:
            Nsc =  (self.epsilon_yd * self.Es * self.Asc) / 1000
        else:
          Nsc = (epsilon_sc * self.Es * self.Asc) / 1000

        # Calculate Nc (Axial force in concrete)
        # Corrected Nc calculation to use x (neutral axis depth) for compression block depth (lambda * x)
        if x < self.D:
          lamda = (x * self.lamda)/ self.D
        else:
          lamda = 1 - ((1-self.lamda)*(self.D/x))
        Nc = (self.eta * (self.fck / self.SFc) * self.b * (self.lamda * x)) / 1000

        # Calculate total Axial force Nd and Moment Md
        Nd = Nst + Nsc + Nc
        # Md formula is retained as per original structure, but its correctness needs further review.
        Md = ((Nsc*(self.D - self.d - self.d_prime))+ (Nc * (self.D - self.d - lamda*self.D*0.5))- (Nd*(self.D*0.5-self.d)))/1000

        return {
            "ε_c": epsilon_c,
            "ε_sc": epsilon_sc,
            "ε_st": epsilon_st,
            "Nst": Nst,
            "Nsc": Nsc,
            "Nc": Nc,
            "Nd": Nd,
            "lamda": lamda,
            "Md": Md
        }
    def compute_domain5_values(self, x):
        # Domain 4: self.D < x < +infinity
        # Strain at extreme compression fiber (top) is epsilon_cu
        epsilon_c = (self.epsilon_cu * self.epsilon_co * x)/((self.epsilon_cu*x)- self.D * (self.epsilon_cu - self.epsilon_co))

        # Strain in compression steel:
        epsilon_sc = (self.epsilon_cu/x)* (x-self.d_prime)

        # Strain in tension steel:
        epsilon_st = (self.epsilon_cu * self.epsilon_co * (x- (self.D - self.d )))/((self.epsilon_cu*x)- self.D * (self.epsilon_cu - self.epsilon_co))

        # Calculate Nst (Axial force in tension steel)
        # Stress in tension steel
        if abs(epsilon_st) > self.epsilon_yd:
          if epsilon_st < 0:
            Nst = -(self.epsilon_yd * self.Es * self.Ast) / 1000
          else:
            Nst =  (self.epsilon_yd * self.Es * self.Ast) / 1000
        else:
          Nst = (epsilon_st * self.Es * self.Ast) / 1000

        # Calculate Nsc (Axial force in compression steel)
        # Stress in compression steel
        if abs(epsilon_sc) > self.epsilon_yd:
          if epsilon_sc < 0:
            Nsc = -(self.epsilon_yd * self.Es * self.Asc) / 1000
          else:
            Nsc =  (self.epsilon_yd * self.Es * self.Asc) / 1000
        else:
          Nsc = (epsilon_sc * self.Es * self.Asc) / 1000

        # Calculate Nc (Axial force in concrete)
        # Corrected Nc calculation to use x (neutral axis depth) for compression block depth (lambda * x)
        if x < self.D:
          lamda = (x * self.lamda)/ self.D
        else:
          lamda = 1 - ((1-self.lamda)*(self.D/x))
        Nc = self.eta * lamda * (self.fck/self.SFc)* self.D * self.b / 1000

        # Calculate total Axial force Nd and Moment Md
        Nd = Nst + Nsc + Nc
        # Md formula is retained as per original structure, but its correctness needs further review.
        Md = ((Nsc*(self.D - self.d - self.d_prime))+ (Nc * (self.D - self.d - lamda *self.D*0.5))- (Nd*(self.D*0.5-self.d)))/1000

        return {
            "ε_c": epsilon_c,
            "ε_sc": epsilon_sc,
            "ε_st": epsilon_st,
            "Nst": Nst,
            "Nsc": Nsc,
            "Nc": Nc,
            "Nd": Nd,
            "lamda": lamda,
            "Md": Md
        }

# Instantiate NeutralAxisDomain with the 'beam' object
na = NeutralAxisDomain(beam)

x = float(input("Enter x (mm): "))

domain = na.get_domain(x)

print("For x =", x, " ---> Domain =", domain)

if domain == "Domain 1":
    vals = na.compute_domain1_values(x)
    print("\nDomain 1 values:")
    print("-" * 40)
    for key, value in vals.items():
        print(f"{key:10s} : {value:15.4f}")
    print("-" * 40)
elif domain == "Domain 2":
    vals = na.compute_domain2_values(x)
    print("\nDomain 2 values:")
    print("-" * 40)
    for key, value in vals.items():
        print(f"{key:10s} : {value:15.4f}")
    print("-" * 40)
elif domain == "Domain 3":
    vals = na.compute_domain3_values(x)
    print("\nDomain 3 values:")
    print("-" * 40)
    for key, value in vals.items():
        print(f"{key:10s} : {value:15.4f}")
    print("-" * 40)
elif domain == "Domain 4":
    vals = na.compute_domain4_values(x)
    print("\nDomain 4 values:")
    print("-" * 40)
    for key, value in vals.items():
        print(f"{key:10s} : {value:15.4f}")
    print("-" * 40)
elif domain == "Domain 4a":
    vals = na.compute_domain4a_values(x)
    print("\nDomain 4a values:")
    print("-" * 40)
    for key, value in vals.items():
        print(f"{key:10s} : {value:15.4f}")
    print("-" * 40)
elif domain == "Domain 5":
    vals = na.compute_domain5_values(x)
    print("\nDomain 5 values:")
    print("-" * 40)
    for key, value in vals.items():
        print(f"{key:10s} : {value:15.4f}")
    print("-" * 40)

Plotting the result in Excel

In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.chart import ScatterChart, Reference, Series
from openpyxl.chart.marker import Marker
from openpyxl.chart.axis import ChartLines # Added this import
from google.colab import files

# IMPORTANT: This assumes you have already defined:
# 1. Your beam object with all required properties
# 2. The NeutralAxisDomain class
# 3. Created the na object: na = NeutralAxisDomain(beam)

def export_domain_analysis_to_excel(na, x_min=-30, x_max=760, x_step=10, filename='domain_analysis_results.xlsx'):
    """
    Export domain analysis results to Excel with graphs

    Parameters:
    -----------
    na : NeutralAxisDomain object
    x_min : float, starting x value (default: -30)
    x_max : float, ending x value (default: 760)
    x_step : float, step interval (default: 10)
    filename : str, output Excel filename
    """

    # Define the range of x values with interval of 10
    x_values = np.arange(x_min, x_max + x_step, x_step)

    print(f"Calculating for {len(x_values)} x values from {x_min} to {x_max}...")

    # Initialize lists to store results
    results = []

    for x in x_values:
        domain = na.get_domain(x)

        # Get values based on domain
        if domain == "Domain 1":
            vals = na.compute_domain1_values(x)
        elif domain == "Domain 2":
            vals = na.compute_domain2_values(x)
        elif domain == "Domain 3":
            vals = na.compute_domain3_values(x)
        elif domain == "Domain 4":
            vals = na.compute_domain4_values(x)
        elif domain == "Domain 4a":
            vals = na.compute_domain4a_values(x)
        elif domain == "Domain 5":
            vals = na.compute_domain5_values(x)

        # Append results
        result_row = {
            'x (mm)': x,
            'Domain': domain,
            'ε_c': vals['ε_c'],
            'ε_sc': vals['ε_sc'],
            'ε_st': vals['ε_st'],
            'Nst (kN)': vals['Nst'],
            'Nsc (kN)': vals['Nsc'],
            'Nc (kN)': vals['Nc'],
            'Nd (kN)': vals['Nd'],
            'lamda': vals['lamda'],
            'Md (kNm)': vals['Md']
        }
        results.append(result_row)

    # Create DataFrame
    df = pd.DataFrame(results)

    # Calculate maximum axial force (Nd_max at M=0)
    Nd_max = ((na.fck / na.SFc) * na.D * na.b + (na.Ast + na.Asc) * na.Es * 0.002) / 1000  # Convert to kN

    # Add Nd_max as a separate row at the bottom
    ndmax_row = {
        'x (mm)': 'N/A',
        'Domain': 'Nd_max',
        'ε_c': '',
        'ε_sc': '',
        'ε_st': '',
        'Nst (kN)': '',
        'Nsc (kN)': '',
        'Nc (kN)': '',
        'Nd (kN)': Nd_max,
        'lamda': '',
        'Md (kNm)': 0.0
    }

    # Append Nd_max row to dataframe
    df = pd.concat([df, pd.DataFrame([ndmax_row])], ignore_index=True)

    # Export to Excel
    print(f"Exporting data to {filename}...")
    df.to_excel(filename, sheet_name='Data', index=False)
    print(f"Nd_max = {Nd_max:.2f} kN at M = 0 kNm (added to bottom of data)")

    print(f"Data exported successfully!")

    # Load workbook to add chart
    print("Creating M-N Interaction Diagram...")
    wb = load_workbook(filename)
    ws = wb['Data']

    # M-N Interaction Diagram (Moment on X-axis, Axial Force on Y-axis)
    chart = ScatterChart()
    chart.title = "M-N Interaction Diagram"
    chart.style = 13
    chart.x_axis.title = "Moment (Md) kN-m"
    chart.y_axis.title = "Axial Force (Nd) kN"
    chart.width = 20
    chart.height = 15

    # Moment (column 11) on X-axis, Axial Force (column 9) on Y-axis
    md_ref = Reference(ws, min_col=11, min_row=2, max_row=ws.max_row)
    nd_ref = Reference(ws, min_col=9, min_row=2, max_row=ws.max_row)
    series = Series(nd_ref, md_ref, title='M-N Interaction Curve')
    series.marker = Marker('circle')
    series.marker.size = 4
    series.graphicalProperties.line.solidFill = '0000FF'
    series.graphicalProperties.line.width = 25000
    chart.series.append(series)

    # Remove legend only
    chart.legend = None

    # Configure axes to show gridlines and better range visibility
    chart.x_axis.majorGridlines = ChartLines() # Changed from True
    chart.y_axis.majorGridlines = ChartLines() # Changed from True
    chart.x_axis.minorGridlines = ChartLines() # Changed from True
    chart.y_axis.minorGridlines = ChartLines() # Changed from True

    # Add axis crosses at zero for better reference
    chart.x_axis.crosses = "autoZero"
    chart.y_axis.crosses = "autoZero"

    ws.add_chart(chart, "M2")

    # Save workbook with chart
    wb.save(filename)
    print(f"Chart added successfully!")

    # Print summary
    print("\n" + "="*60)
    print("SUMMARY OF ANALYSIS")
    print("="*60)
    print(f"Total data points: {len(df)}")
    print(f"\nDomain distribution:")
    print(df['Domain'].value_counts().sort_index())
    print(f"\nKey boundary values:")
    print(f"x1 (Domain 2-3 boundary): {na.x1:.2f} mm")
    print(f"x2 (Domain 3-4 boundary): {na.x2:.2f} mm")
    print(f"D-d (Domain 4-4a boundary): {na.D - na.d:.2f} mm")
    print(f"D (Domain 4a-5 boundary): {na.D:.2f} mm")
    print("\n" + "="*60)
    print(f"\nRange of Nd: {df['Nd (kN)'].min():.2f} to {df['Nd (kN)'].max():.2f} kN")
    print(f"Range of Md: {df['Md (kNm)'].min():.2f} to {df['Md (kNm)'].max():.2f} kNm")
    print(f"Maximum Moment: {df.loc[df['Md (kNm)'].idxmax(), 'Md (kNm)']:.2f} kNm at x = {df.loc[df['Md (kNm)'].idxmax(), 'x (mm)']:.2f} mm")
    print(f"\nMaximum Axial Force (Nd_max at M=0): {Nd_max:.2f} kN")
    print("="*60)
    print(f"\nExcel file '{filename}' created successfully!")

    # Download the file in Google Colab
    print(f"\nDownloading {filename}...")
    files.download(filename)
    print("✓ Download initiated!")

    return df


# USAGE EXAMPLE:
# After defining your beam object and creating na = NeutralAxisDomain(beam),
# simply call:
#
# df_results = export_domain_analysis_to_excel(na)
#
# Or with custom parameters:
# df_results = export_domain_analysis_to_excel(na, x_min=-30, x_max=760, x_step=10, filename='my_results.xlsx')


if __name__ == "__main__":
    # This section will only run if you execute this script directly
    # Make sure to have 'beam' and 'na' defined before this point

    try:
        # Try to export (this assumes na is already defined)
        df_results = export_domain_analysis_to_excel(na, x_min=-30, x_max=760, x_step=10)
        print("\n✓ Export completed successfully!")
    except NameError:
        print("\n⚠ Please make sure to:")
        print("1. Define your 'beam' object with all required properties")
        print("2. Create 'na = NeutralAxisDomain(beam)' before running this script")
        print("\nThen call: export_domain_analysis_to_excel(na)")